<a href="https://colab.research.google.com/github/kalavagunta-vamshi/Fine-tuning-BERT-with-LoRA-quantization-technique/blob/main/Fine_Tuned_BERT_for_Sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate -U
!pip install peft

In [ ]:
# Import necessary libraries
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:

# Load and prepare the dataset
imdb_dataset = load_dataset("imdb")

N = 500

rand_idx = np.random.randint(24999, size=N)

x_train = imdb_dataset['train'][rand_idx]['text']
y_train = imdb_dataset['train'][rand_idx]['label']

x_test = imdb_dataset['test'][rand_idx]['text']
y_test = imdb_dataset['test'][rand_idx]['label']

dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
                             'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

In [ ]:
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.48

In [ ]:
# Set up model

model_checkpoint = 'distilbert-base-uncased'
# model_checkpoint = 'roberta-base'

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# Set up tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Define tokenization function
def tokenize_function(examples):
    text = examples["text"]
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [ ]:
# Set up data collator and evaluation metric
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [ ]:
sentiment_examples = [
    "The film's innovative cinematography left me in awe throughout.",
    "A disappointing sequel that fails to capture the magic of the original.",
    "The lead actor's performance was nothing short of Oscar-worthy.",
    "This movie is a complete disaster; avoid at all costs.",
    "An emotional rollercoaster that kept me engaged from start to finish.",
    "The dialogue was so cringeworthy, I couldn't make it past the first act.",
    "A masterpiece of modern cinema that will be studied for years to come.",
    "The special effects were impressive, but couldn't save the weak storyline.",
    "This film tackles complex issues with sensitivity and nuance.",
    "A pretentious art house film that tries too hard to be profound.",
    "The chemistry between the lead actors was palpable and believable.",
    "This movie is so bad, it's almost good... almost.",
    "A thrilling ride that keeps you guessing until the very end.",
    "The pacing was so slow, I found myself checking my watch repeatedly.",
    "A heartwarming tale that reminds us of the power of human connection.",
    "This film is a textbook example of style over substance.",
    "The soundtrack perfectly complemented the on-screen action.",
    "A confusing mess of a plot that left me more frustrated than entertained.",
    "This movie pushes the boundaries of its genre in exciting new ways.",
    "A formulaic rom-com that brings nothing new to the table."
]


print("Untrained model predictions:")
print("----------------------------")
for text in sentiment_examples:
    inputs = tokenizer.encode(text, return_tensors="pt")
    logits = model(inputs).logits
    predictions = torch.argmax(logits)
    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
The film's innovative cinematography left me in awe throughout. - Negative
A disappointing sequel that fails to capture the magic of the original. - Negative
The lead actor's performance was nothing short of Oscar-worthy. - Negative
This movie is a complete disaster; avoid at all costs. - Negative
An emotional rollercoaster that kept me engaged from start to finish. - Negative
The dialogue was so cringeworthy, I couldn't make it past the first act. - Negative
A masterpiece of modern cinema that will be studied for years to come. - Negative
The special effects were impressive, but couldn't save the weak storyline. - Negative
This film tackles complex issues with sensitivity and nuance. - Negative
A pretentious art house film that tries too hard to be profound. - Negative
The chemistry between the lead actors was palpable and believable. - Negative
This movie is so bad, it's almost good... almost. - Negative
A thrilling ride that 

In [ ]:


sentiment_examples_with_category = {
    "The film's innovative cinematography left me in awe throughout.": "Positive",
    "A disappointing sequel that fails to capture the magic of the original.": "Negative",
    "The lead actor's performance was nothing short of Oscar-worthy.": "Positive",
    "This movie is a complete disaster; avoid at all costs.": "Negative",
    "An emotional rollercoaster that kept me engaged from start to finish.": "Positive",
    "The dialogue was so cringeworthy, I couldn't make it past the first act.": "Negative",
    "A masterpiece of modern cinema that will be studied for years to come.": "Positive",
    "The special effects were impressive, but couldn't save the weak storyline.": "Negative",
    "This film tackles complex issues with sensitivity and nuance.": "Positive",
    "A pretentious art house film that tries too hard to be profound.": "Negative",
    "The chemistry between the lead actors was palpable and believable.": "Positive",
    "This movie is so bad, it's almost good... almost.": "Negative",
    "A thrilling ride that keeps you guessing until the very end.": "Positive",
    "The pacing was so slow, I found myself checking my watch repeatedly.": "Negative",
    "A heartwarming tale that reminds us of the power of human connection.": "Positive",
    "This film is a textbook example of style over substance.": "Negative",
    "The soundtrack perfectly complemented the on-screen action.": "Positive",
    "A confusing mess of a plot that left me more frustrated than entertained.": "Negative",
    "This movie pushes the boundaries of its genre in exciting new ways.": "Positive",
    "A formulaic rom-com that brings nothing new to the table.": "Negative"
}


In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [ ]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [ ]:
lr = 1e-3
batch_size = 4
num_epochs = 1

In [ ]:
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-23-ea83d2950a86>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.411567,{'accuracy': 0.854}


Trainer is attempting to log a value of "{'accuracy': 0.854}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=125, training_loss=0.46163116455078124, metrics={'train_runtime': 2046.6978, 'train_samples_per_second': 0.244, 'train_steps_per_second': 0.061, 'total_flos': 59640902771808.0, 'train_loss': 0.46163116455078124, 'epoch': 1.0})

In [ ]:
print("Trained model predictions:")
print("--------------------------")
for text in sentiment_examples:
    inputs = tokenizer.encode(text, return_tensors="pt")
    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices
    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
The film's innovative cinematography left me in awe throughout. - Positive
A disappointing sequel that fails to capture the magic of the original. - Negative
The lead actor's performance was nothing short of Oscar-worthy. - Positive
This movie is a complete disaster; avoid at all costs. - Negative
An emotional rollercoaster that kept me engaged from start to finish. - Positive
The dialogue was so cringeworthy, I couldn't make it past the first act. - Negative
A masterpiece of modern cinema that will be studied for years to come. - Positive
The special effects were impressive, but couldn't save the weak storyline. - Negative
This film tackles complex issues with sensitivity and nuance. - Positive
A pretentious art house film that tries too hard to be profound. - Negative
The chemistry between the lead actors was palpable and believable. - Positive
This movie is so bad, it's almost good... almost. - Negative
A thrilling ride that keep

In [ ]:


sentiment_examples_with_category = {
    "The film's innovative cinematography left me in awe throughout.": "Positive",
    "A disappointing sequel that fails to capture the magic of the original.": "Negative",
    "The lead actor's performance was nothing short of Oscar-worthy.": "Positive",
    "This movie is a complete disaster; avoid at all costs.": "Negative",
    "An emotional rollercoaster that kept me engaged from start to finish.": "Positive",
    "The dialogue was so cringeworthy, I couldn't make it past the first act.": "Negative",
    "A masterpiece of modern cinema that will be studied for years to come.": "Positive",
    "The special effects were impressive, but couldn't save the weak storyline.": "Negative",
    "This film tackles complex issues with sensitivity and nuance.": "Positive",
    "A pretentious art house film that tries too hard to be profound.": "Negative",
    "The chemistry between the lead actors was palpable and believable.": "Positive",
    "This movie is so bad, it's almost good... almost.": "Negative",
    "A thrilling ride that keeps you guessing until the very end.": "Positive",
    "The pacing was so slow, I found myself checking my watch repeatedly.": "Negative",
    "A heartwarming tale that reminds us of the power of human connection.": "Positive",
    "This film is a textbook example of style over substance.": "Negative",
    "The soundtrack perfectly complemented the on-screen action.": "Positive",
    "A confusing mess of a plot that left me more frustrated than entertained.": "Negative",
    "This movie pushes the boundaries of its genre in exciting new ways.": "Positive",
    "A formulaic rom-com that brings nothing new to the table.": "Negative"
}
